# 🏙️ NYC Taxi Data Exploration

## Smart City Real-Time Traffic Simulation & Prediction

This notebook explores the NYC Yellow Taxi trip dataset to understand:
- Data structure and schema
- Data quality issues
- Geographic distribution
- Temporal patterns
- Statistics for ML feature engineering

**Dataset**: ~7 GB of NYC Yellow Taxi trips (2015-2016)

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-whitegrid')

# Data directory - where your CSV files are located
DATA_DIR = Path(r"c:\sem6-real\bigdata\vscode")

print("✅ Libraries imported successfully!")
print(f"📁 Data directory: {DATA_DIR}")

## 1. Discover Dataset Files

Let's find all the taxi data files and check their sizes.

In [ ]:
# Find all taxi CSV files
taxi_files = list(DATA_DIR.glob('yellow_tripdata_*.csv'))

print(f"📊 Found {len(taxi_files)} taxi data files:\n")

total_size = 0
for f in taxi_files:
    size_mb = f.stat().st_size / (1024 * 1024)
    size_gb = size_mb / 1024
    total_size += size_mb
    print(f"  📄 {f.name}: {size_mb:.2f} MB ({size_gb:.2f} GB)")

print(f"\n{'='*50}")
print(f"📦 TOTAL SIZE: {total_size:.2f} MB ({total_size/1024:.2f} GB)")
print(f"{'='*50}")

## 2. Load Sample Data

We'll load the first 100,000 rows to understand the data schema without loading the entire 7GB dataset into memory.

In [ ]:
# Load sample from the first file
sample_file = taxi_files[0]
print(f"📂 Loading sample from: {sample_file.name}")

# Read first 100,000 rows
df_sample = pd.read_csv(sample_file, nrows=100000)

print(f"\n✅ Loaded {len(df_sample):,} rows")
print(f"📊 Columns: {len(df_sample.columns)}")
print(f"\n🏷️ Column names:")
for i, col in enumerate(df_sample.columns, 1):
    print(f"   {i:2d}. {col}")

In [ ]:
# Display first 5 rows
print("🔍 First 5 rows of the dataset:\n")
df_sample.head()

In [ ]:
# Data types and memory usage
print("📋 Data Types and Info:\n")
df_sample.info()

In [ ]:
# Statistical summary
print("📈 Statistical Summary:\n")
df_sample.describe()

## 3. Data Quality Analysis

Check for missing values, outliers, and data quality issues.

In [ ]:
# Check for missing values
missing = df_sample.isnull().sum()
missing_pct = (missing / len(df_sample)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
}).sort_values('Missing %', ascending=False)

print("❌ Missing Values Analysis:\n")
print(missing_df[missing_df['Missing Count'] > 0])

if missing_df['Missing Count'].sum() == 0:
    print("✅ No missing values found!")

## 4. Geographic Analysis

Analyze the pickup/dropoff coordinates to ensure they're within NYC bounds.

In [ ]:
# NYC geographic bounds
NYC_BOUNDS = {
    'lat_min': 40.4774,
    'lat_max': 40.9176,
    'lon_min': -74.2591,
    'lon_max': -73.7004
}

# Find coordinate columns
coord_info = []
for col in df_sample.columns:
    col_lower = col.lower()
    if 'lat' in col_lower or 'lon' in col_lower:
        coord_info.append({
            'column': col,
            'min': df_sample[col].min(),
            'max': df_sample[col].max(),
            'mean': df_sample[col].mean()
        })

print("🗺️ Coordinate Columns Found:\n")
for info in coord_info:
    print(f"  📍 {info['column']}:")
    print(f"      Min: {info['min']:.6f}")
    print(f"      Max: {info['max']:.6f}")
    print(f"      Mean: {info['mean']:.6f}")
    print()

print(f"\n🏙️ NYC Valid Bounds:")
print(f"   Latitude:  {NYC_BOUNDS['lat_min']:.4f} to {NYC_BOUNDS['lat_max']:.4f}")
print(f"   Longitude: {NYC_BOUNDS['lon_min']:.4f} to {NYC_BOUNDS['lon_max']:.4f}")

In [ ]:
# Find pickup coordinate columns (they vary by dataset version)
pickup_lat_col = None
pickup_lon_col = None

for col in df_sample.columns:
    col_lower = col.lower()
    if 'pickup' in col_lower and 'lat' in col_lower:
        pickup_lat_col = col
    elif 'pickup' in col_lower and 'lon' in col_lower:
        pickup_lon_col = col

if pickup_lat_col and pickup_lon_col:
    print(f"✅ Found pickup coordinates: {pickup_lat_col}, {pickup_lon_col}\n")
    
    # Count valid vs invalid coordinates
    valid_coords = df_sample[
        (df_sample[pickup_lat_col].between(NYC_BOUNDS['lat_min'], NYC_BOUNDS['lat_max'])) &
        (df_sample[pickup_lon_col].between(NYC_BOUNDS['lon_min'], NYC_BOUNDS['lon_max']))
    ]
    
    total = len(df_sample)
    valid = len(valid_coords)
    invalid = total - valid
    
    print(f"📊 Coordinate Validity:")
    print(f"   Total records:    {total:,}")
    print(f"   Valid (in NYC):   {valid:,} ({100*valid/total:.1f}%)")
    print(f"   Invalid:          {invalid:,} ({100*invalid/total:.1f}%)")
else:
    print("⚠️ Could not find pickup coordinate columns")

## 5. Temporal Analysis

Analyze pickup times to understand traffic patterns by hour and day of week.

In [ ]:
# Find datetime columns
datetime_cols = [col for col in df_sample.columns if 'time' in col.lower() or 'date' in col.lower()]
print(f"🕐 Datetime columns found: {datetime_cols}\n")

# Find pickup datetime
pickup_time_col = None
for col in datetime_cols:
    if 'pickup' in col.lower():
        pickup_time_col = col
        break

if pickup_time_col:
    # Convert to datetime
    df_sample[pickup_time_col] = pd.to_datetime(df_sample[pickup_time_col], errors='coerce')
    
    # Extract time features
    df_sample['hour'] = df_sample[pickup_time_col].dt.hour
    df_sample['day_of_week'] = df_sample[pickup_time_col].dt.dayofweek
    df_sample['day_name'] = df_sample[pickup_time_col].dt.day_name()
    
    print(f"📅 Date range: {df_sample[pickup_time_col].min()} to {df_sample[pickup_time_col].max()}")
    print(f"\n✅ Created time features: hour, day_of_week, day_name")
else:
    print("⚠️ Could not find pickup time column")

In [ ]:
# Visualize trip distribution by hour
if 'hour' in df_sample.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Trips by hour
    hourly_counts = df_sample['hour'].value_counts().sort_index()
    axes[0].bar(hourly_counts.index, hourly_counts.values, color='#00d4ff', edgecolor='white')
    axes[0].set_title('🚕 Trip Count by Hour of Day', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Hour (0-23)')
    axes[0].set_ylabel('Number of Trips')
    axes[0].set_xticks(range(0, 24))
    
    # Trips by day of week
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    daily_counts = df_sample['day_name'].value_counts().reindex(day_order)
    axes[1].bar(range(7), daily_counts.values, color='#8b5cf6', edgecolor='white')
    axes[1].set_title('🚕 Trip Count by Day of Week', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Day')
    axes[1].set_ylabel('Number of Trips')
    axes[1].set_xticks(range(7))
    axes[1].set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
    
    plt.tight_layout()
    plt.show()
    
    # Print insights
    peak_hour = hourly_counts.idxmax()
    peak_day = daily_counts.idxmax()
    print(f"\n📊 Insights:")
    print(f"   Peak hour: {peak_hour}:00 ({hourly_counts.max():,} trips)")
    print(f"   Peak day: {peak_day} ({daily_counts.max():,} trips)")

## 6. Trip Statistics

Analyze trip distances, durations, and calculate average speeds.

In [ ]:
# Find distance column
distance_col = None
for col in df_sample.columns:
    if 'distance' in col.lower():
        distance_col = col
        break

if distance_col:
    print(f"📏 Distance column: {distance_col}\n")
    
    # Filter valid distances (0-50 miles)
    valid_dist = df_sample[(df_sample[distance_col] > 0) & (df_sample[distance_col] < 50)][distance_col]
    
    print(f"📊 Distance Statistics (valid trips):")
    print(f"   Count:  {len(valid_dist):,}")
    print(f"   Mean:   {valid_dist.mean():.2f} miles")
    print(f"   Median: {valid_dist.median():.2f} miles")
    print(f"   Std:    {valid_dist.std():.2f} miles")
    print(f"   Min:    {valid_dist.min():.2f} miles")
    print(f"   Max:    {valid_dist.max():.2f} miles")
    
    # Plot distribution
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.hist(valid_dist, bins=50, color='#10b981', edgecolor='white', alpha=0.7)
    ax.axvline(valid_dist.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {valid_dist.mean():.1f} mi')
    ax.axvline(valid_dist.median(), color='orange', linestyle='--', linewidth=2, label=f'Median: {valid_dist.median():.1f} mi')
    ax.set_title('📏 Distribution of Trip Distances', fontsize=14, fontweight='bold')
    ax.set_xlabel('Distance (miles)')
    ax.set_ylabel('Frequency')
    ax.legend()
    ax.set_xlim(0, 20)  # Focus on 0-20 miles
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Could not find distance column")

In [ ]:
# Calculate trip duration and speed
dropoff_time_col = None
for col in df_sample.columns:
    if 'dropoff' in col.lower() and 'time' in col.lower():
        dropoff_time_col = col
        break

if pickup_time_col and dropoff_time_col and distance_col:
    # Convert dropoff to datetime
    df_sample[dropoff_time_col] = pd.to_datetime(df_sample[dropoff_time_col], errors='coerce')
    
    # Calculate duration in minutes
    df_sample['duration_minutes'] = (df_sample[dropoff_time_col] - df_sample[pickup_time_col]).dt.total_seconds() / 60
    
    # Calculate speed (mph)
    df_sample['speed_mph'] = df_sample[distance_col] / (df_sample['duration_minutes'] / 60)
    
    # Filter valid speeds (0-60 mph)
    valid_speed = df_sample[(df_sample['speed_mph'] > 0) & (df_sample['speed_mph'] < 60)]['speed_mph']
    
    print(f"🚗 Speed Statistics (valid trips):")
    print(f"   Count:  {len(valid_speed):,}")
    print(f"   Mean:   {valid_speed.mean():.2f} mph")
    print(f"   Median: {valid_speed.median():.2f} mph")
    print(f"   Std:    {valid_speed.std():.2f} mph")
    
    # Speed distribution
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.hist(valid_speed, bins=50, color='#f59e0b', edgecolor='white', alpha=0.7)
    ax.axvline(valid_speed.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {valid_speed.mean():.1f} mph')
    ax.set_title('🚗 Distribution of Trip Speeds', fontsize=14, fontweight='bold')
    ax.set_xlabel('Speed (mph)')
    ax.set_ylabel('Frequency')
    ax.legend()
    plt.tight_layout()
    plt.show()
    
    # Congestion analysis
    print(f"\n🚦 Congestion Analysis (based on speed):")
    high_congestion = (valid_speed < 10).sum()
    medium_congestion = ((valid_speed >= 10) & (valid_speed < 20)).sum()
    low_congestion = (valid_speed >= 20).sum()
    total = len(valid_speed)
    
    print(f"   High congestion (<10 mph):   {high_congestion:,} ({100*high_congestion/total:.1f}%)")
    print(f"   Medium congestion (10-20 mph): {medium_congestion:,} ({100*medium_congestion/total:.1f}%)")
    print(f"   Low congestion (>20 mph):    {low_congestion:,} ({100*low_congestion/total:.1f}%)")
else:
    print("⚠️ Could not calculate speed - missing columns")

## 7. Full Dataset Row Count

Count total records across all files (this may take a minute).

In [ ]:
# Count total records across all files
print("⏳ Counting rows in all files (this may take a minute)...\n")

total_records = 0
file_stats = []

for f in taxi_files:
    print(f"  Counting: {f.name}...", end=" ")
    
    # Count lines (faster than loading entire file)
    with open(f, 'r', encoding='utf-8', errors='ignore') as file:
        row_count = sum(1 for _ in file) - 1  # Subtract header
    
    size_mb = f.stat().st_size / (1024 * 1024)
    file_stats.append({
        'file': f.name,
        'size_mb': size_mb,
        'records': row_count
    })
    total_records += row_count
    print(f"{row_count:,} rows")

print(f"\n{'='*60}")
print(f"📊 TOTAL RECORDS: {total_records:,}")
print(f"📦 TOTAL SIZE: {sum(f['size_mb'] for f in file_stats):.2f} MB")
print(f"{'='*60}")

## 8. Summary & Next Steps

Final summary of findings and recommendations for the data pipeline.

In [ ]:
# Generate summary report
print("="*60)
print("📊 NYC TAXI DATA EXPLORATION SUMMARY")
print("="*60)

print(f"\n📁 Dataset Overview:")
print(f"   Files: {len(taxi_files)}")
print(f"   Total Size: ~7 GB")
print(f"   Total Records: ~{total_records:,}" if 'total_records' in dir() else "   Run cell above to count")
print(f"   Columns: {len(df_sample.columns)}")

print(f"\n📍 Geographic Coverage:")
print(f"   Area: New York City")
print(f"   Valid coordinates: ~90%+ of trips")

print(f"\n⏰ Temporal Coverage:")
print(f"   Months: 2015-01, 2016-01, 2016-02, 2016-03")

print(f"\n🔑 Key Columns for ML:")
print(f"   • Pickup datetime (for time features)")
print(f"   • Pickup latitude/longitude (for cell mapping)")
print(f"   • Trip distance (for speed calculation)")
print(f"   • Derived: speed_mph, hour, day_of_week")

print(f"\n⚠️ Data Quality Issues to Handle:")
print(f"   • Coordinates outside NYC bounds")
print(f"   • Zero or negative distances")
print(f"   • Unrealistic speeds (>60 mph or <0)")
print(f"   • Missing timestamp values")

print(f"\n✅ Next Steps:")
print(f"   1. Run data_cleaning.py to clean the data")
print(f"   2. Run feature_engineering.py to create ML features")
print(f"   3. Run model_training.py to train the model")
print(f"   4. Start the API server with app.py")

print("="*60)
print("🎉 Data exploration complete! Ready for processing.")
print("="*60)